In [1]:
# General SVM Class that will be a base for all the other SVM Implementations
import torch
import numpy as np

class SVM(object):
    def __init__(self):
        self.W = None

        
    def train(self, X_train, y_train, learning_rate=1e-3, reg=1e-5, num_iters=100,
            batch_size=200, verbose=False):
        """
            Takes in the training data and labels as well as training parameters.
            Updates the weights using stochastic gradient descent
        """
        self.W, loss_history = train_linear_classifier(self.loss, self.W, X_train, y_train, learning_rate, reg, num_iters, batch_size, verbose)
        
        return loss_history
        
    def predict(self, X):
        """
            Takes in the test data and outputs a prediction torch
        """
        
        
    def loss(self, W, X_batch, y_batch, reg):
    """
    Compute the loss function and its derivative.
    Subclasses will override this.

    Inputs:
    - W: A PyTorch tensor of shape (D, C) containing (trained) weight of a model.
    - X_batch: A PyTorch tensor of shape (N, D) containing a minibatch of N
      data points; each point has dimension D.
    - y_batch: A PyTorch tensor of shape (N,) containing labels for the minibatch.
    - reg: (float) regularization strength.

    Returns: A tuple containing:
    - loss as a single float
    - gradient with respect to self.W; an tensor of the same shape as W
    """
    raise NotImplementedError

    def sample_batch(X, y, num_train, batch_size):
        # randomly sample "batch_size" (Xi,yi) in (X,y)

        inx = torch.randint(0, num_train, (batch_size))

        X_batch = X[inx, :]
        y_batch = y[inx]

        return X_batch, y_batch

    def train_linear_classifier(loss_fun, W, X, y, learning_rate=1e-3, reg=1e-5, num_iters=100,
            batch_size=200, verbose=False):
        num_train, dim = X.shape

        # initialize weight
        if W is None:
            num_class = torch.max(y)+1

            W = torch.zeros([dim, num_class], dtype=X.dtype)
            W = torch.zeros([dim+1, num_class], dtype=X.dtype)
        else:
            num_class = W.shape[1]

        # Stochastic Gradient Descent
        loss_history = []
        for it in xrange(num_iters):
            # sample batch
            X_batch, y_batch = sample_batch(X, y, num_train, batch_size)

            # compute loss and gradient
            loss, grad = loss_fun(X_batch, y_batch, reg)
            loss_history.append(loss)

            # update weight
            W = W - learning_rate*grad

        # return 
        return W, loss_history
    
class LinearSVM(SVM):
    def loss(self, W, X_batch, y_batch, reg):
    """
    Compute the loss function and its derivative.
    Subclasses will override this.

    Inputs:
    - W: A PyTorch tensor of shape (D, C) containing (trained) weight of a model.
    - X_batch: A PyTorch tensor of shape (N, D) containing a minibatch of N
      data points; each point has dimension D.
    - y_batch: A PyTorch tensor of shape (N,) containing labels for the minibatch.
    - reg: (float) regularization strength.

    Returns: A tuple containing:
    - loss as a single float
    - gradient with respect to self.W; an tensor of the same shape as W
    """
    

SyntaxError: invalid syntax (<ipython-input-1-9014a466bc9d>, line 16)

# Note

* Do we use bias? If so, where does the bias is append?
* Not sure why the template seperate train into train_linear_classifier